# Detector layout

Edited by Louise, 09-11-2020

This notebook explains how the QUBIC detector layout is defined in QUBIC soft. 

In [ ]:
from __future__ import division, print_function

%matplotlib inline
%matplotlib notebook

import numpy as np
from astropy.io import fits
import matplotlib.pyplot as plt

from qubicpack.utilities import Qubic_DataDir
import qubic

plt.rcParams['figure.figsize'] = (12, 6)

In [ ]:
# Use a tool from qubicpack to get a path
basedir = Qubic_DataDir(datafile='instrument.py', ) 
print('basedir : ', basedir)

# Get a dictionary
dictfilename = basedir + '/dicts/global_source_oneDet.dict'
d = qubic.qubicdict.qubicDict()
d.read_from_file(dictfilename)
print(d['detarray'])

d['MultiBand'] = False
d['nf_sub'] = 1

# Choose TD or FI
config = 'TD'
d['config'] = config

## Look at the DetArray calibration file

It is a .fits. There is one for the TD and one for the FI. 

In [ ]:
hdu = fits.open(basedir + 'calfiles/CalQubic_DetArray_v4_' + config + '.fits')
print(hdu.info())



Look at the header:

In [ ]:
hdu[0].header

The frame used is ONAFP:

In [ ]:
hdu[0].header['FRAME']

In [ ]:
# Center of each bolometer
center = hdu['center'].data
x = center[:, :, 0]
y = center[:, :, 1]

# Corners of each bolo
corner = hdu['corner'].data

quadrant = hdu['quadrant'].data
ordering = hdu['ordering'].data
removed = hdu['removed'].data
efficiency = hdu['efficiency'].data

plt.subplots(2, 2, figsize=(12, 10))
plt.suptitle('Detector Array Calibration file defined in ONAFP.')
plt.subplot(221)
plt.pcolor(x, y, quadrant)
plt.title('Quadrant')
plt.xlabel('X_ONAFP')
plt.ylabel('Y_ONAFP')
plt.colorbar()

plt.subplot(222)
plt.pcolor(x, y, ordering)
plt.title('Ordering')
plt.xlabel('X_ONAFP')
plt.ylabel('Y_ONAFP')
plt.colorbar()

plt.subplot(223)
plt.pcolor(x, y, removed)
plt.title('Removed')
plt.xlabel('X_ONAFP')
plt.ylabel('Y_ONAFP')
plt.colorbar()

plt.subplot(224)
plt.pcolor(x, y, efficiency)
plt.title('Efficiency')
plt.xlabel('X_ONAFP')
plt.ylabel('Y_ONAFP')
plt.colorbar()

In [ ]:
# vertex contains the coordinates of the 4 TES corners. 
vertex = hdu['corner'].data
print(vertex.shape)

plt.figure(figsize=(6, 6))
for i in range(4):
    xvertex = vertex[:, :, i, 0].flatten()
    yvertex = vertex[:, :, i, 1].flatten()
    plt.scatter(xvertex, yvertex, marker='.')
    plt.title('TES Corners')
    plt.xlabel('X_ONAFP [m]')
    plt.ylabel('Y_ONAFP [m]')

## A simple example using `PackedTable` from `pysimulators`

The instanciation of the detector layout in the Qubic soft is done with a `PackedTable` object.

The documentation is very well written in `core.py`. I just reproduce it here.

The PackedTable class gathers information from a set of elements which can have a multi-dimensional layout. This information can transparently be accessed as packed or unpacked arrays.

Example
-------
Let's consider a 3x3 array of detectors, in which the top-left detector
is not working. We will define a mask to flag this detector.

In [ ]:
from pysimulators import PackedTable
selection = [[True, True, False],
             [True, True, True],
             [True, True, True]]
gain = [[1.0, 1.2, 1.5],
        [0.9, 1.0, 1.0],
        [0.8, 1.0, 1.0]]

table = PackedTable((3, 3), selection=selection, gain=gain)

Only the values for the selected detectors are stored, in 1-dimensional arrays:

In [ ]:
table.gain

But the 2-dimensional table can be recovered:

In [ ]:
table.all.gain

The number of selected detectors is:

In [ ]:
len(table)

and the number of all detectors is:

In [ ]:
len(table.all)

Now, let's have a more complex example: an array of detectors made of 4 identical 3x3 subarrays in which one corner detector is blind and for which we will define an indexing scheme. The first subarray is placed on the upper right quadrant and the position of the other arrays is obtained by rotating the first array by 90, 180 and 270 degrees.

A natural indexing, the one used in the Calfiles for the QUBIC focal plane, would be given by:

In [ ]:
ordering = [[-1, 14, 17,  0,  1, -1],
            [10, 13, 16,  3,  4,  5],
            [ 9, 12, 15,  6,  7,  8],
            [26, 25, 24, 33, 30, 27],
            [23, 22, 21, 34, 31, 28],
            [-1, 19, 18, 35, 32, -1]]

The following mask only keeps the 2 subarrays on the left:

In [ ]:
selection = [[ True,  True,  True, False, False, False],
             [ True,  True,  True, False, False, False],
             [ True,  True,  True, False, False, False],
             [ True,  True,  True, False, False, False],
             [ True,  True,  True, False, False, False],
             [ True,  True,  True, False, False, False]]

table = PackedTable((6, 6), selection=selection, ordering=ordering)

Then, the numbering of the table fields follows the list of selected indices stored in:

In [ ]:
print(table.index)

which are the 1d-collapsed indices of the following array coordinates:

In [ ]:
print([(i // 6, i % 6) for i in table.index])

In [ ]:
plt.subplots(1, 3)
plt.subplot(131)
plt.imshow(ordering)
plt.title('Ordering')
plt.colorbar()

plt.subplot(132)
plt.imshow(selection)
plt.title('Selection')
plt.colorbar()

plt.subplot(133)
plt.imshow(table.all.index)
plt.title('Index')
plt.colorbar()

## Get the same from a Qubic Instrument

The same information (center, efficiency, vertex=corners, removed...) and more are accessible with ```q.detector.``` which is a `Layout` object.

When you initialize a QubicInstrument, the detector layout is built with `_get_detector_layout()` from `instrument.py`. This function reads the calibration fits file with:

`shape, vertex, removed, index, quadrant, efficiency = self.calibration.get('detarray')`
This method is defined in `calibration.py`. In this function, we make a pi/2 rotation from ONAFP to GRF frame. 

!!! **The calibration file is defined in the ONAFP frame. However, QUBIC soft works in the GRF frame. For this reason there is a pi/2 rotation between the following plots and the previous ones from the calibration file.** 

Then, it instanciates a `Layout()` object of `pysimulators` (see `layouts.py`) which is actually a class that inherites from the `PackedTable()` we studied above.



In [ ]:
# Make a QUBIC intrument
q = qubic.QubicInstrument(d)

# Look at what q.detector contains
from pprint import pprint
pprint(vars(q.detector))

The `Layout()` class provides a method to plot the detector array:

In [ ]:
plt.figure(figsize=(6,4))
q.detector.plot(fill=True)
plt.title('Detector Array in the GRF frame')
plt.xlabel('X_GRF')
plt.ylabel('Y_GRF')

#### Look at the quadrants: 

!!!! Difference between ```q.detector.all.``` and `q.detector.`:

   - `q.detector.` is always a 1D array where only the selected detectors are saved. For example, for the TD, you will have only 248 elements.

   - `q.detector.all` is a 2D array (34x34) with all the detectors.

So you can plot the same using `plt.scatter` or `plt.pcolor`.

In [ ]:
xGRF = q.detector.center[:, 0]
yGRF = q.detector.center[:, 1]
quadrant = q.detector.quadrant
print(xGRF.shape, quadrant.shape)

xGRF_2D = q.detector.all.center[:, :, 0]
yGRF_2D = q.detector.all.center[:, :, 1]
quadrant_2D = q.detector.all.quadrant
print(xGRF_2D.shape, quadrant_2D.shape)

plt.subplots(1, 2, figsize=(10, 4))

plt.subplot(121)
plt.scatter(xGRF, yGRF, c=quadrant, s=100, marker='s')
plt.title('Index q.detector')
plt.xlabel('X_GRF')
plt.ylabel('Y_GRF')
plt.colorbar()

plt.subplot(122)
plt.pcolor(xGRF_2D, yGRF_2D, quadrant_2D)
plt.title('Index q.detector.all')
plt.xlabel('X_GRF')
plt.ylabel('Y_GRF')
plt.colorbar()

#### Detector indices

In [ ]:
index = q.detector.index
index_2D = q.detector.all.index

plt.subplots(1, 2, figsize=(11, 4))

plt.subplot(121)
plt.scatter(xGRF, yGRF, c=index, s=100, marker='s')
plt.title('Index with q.detector')
plt.xlabel('X_GRF')
plt.ylabel('Y_GRF')
plt.colorbar()

plt.subplot(122)
plt.pcolor(xGRF_2D, yGRF_2D, index_2D)
plt.title('Index with q.detector.all')
plt.xlabel('X_GRF')
plt.ylabel('Y_GRF')
plt.colorbar()

#### Back to the ONAFP frame

There is a $-pi/2$ rotation from $R_{GRF}$ to $R_{ONAFP}$ where $R$ means the referential. So we must apply a $+pi/2$ rotation on the $(x, y)_{GRF}$ coordinates to get the $(x, y)_{ONAFP}$ coordinates:

In [ ]:
xONAFP = - yGRF
yONAFP = xGRF

xONAFP_2D = - yGRF_2D
yONAFP_2D = xGRF_2D

In [ ]:
plt.subplots(1, 2, figsize=(11, 4))

plt.subplot(121)
plt.scatter(xONAFP, yONAFP, c=index, s=100, marker='s')
plt.title('Index with q.detector')
plt.xlabel('X_ONAFP')
plt.ylabel('Y_ONAFP')
plt.colorbar()

plt.subplot(122)
plt.pcolor(xONAFP_2D, yONAFP_2D, index_2D)
plt.title('Index with q.detector.all')
plt.xlabel('X_ONAFP')
plt.ylabel('Y_ONAFP')
plt.colorbar()

In [ ]:
plt.figure()
plt.pcolor(xONAFP_2D, yONAFP_2D, q.detector.all.quadrant)
plt.colorbar()